In [1]:
import os
import itk
import vtk
from vtk import vtkCommand
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DICOM_PATH = "/Users/benjaminhon/Developer/HeadHunter/notebooks/220259"
NII_PATH = "/Users/benjaminhon/Developer/HeadHunter/notebooks/220259.nii"
SLICE = 7

# ITK Types

In [3]:
IF3 = itk.Image[itk.F, 3]
IUC3 = itk.Image[itk.UC, 3]
IRGBUC3 = itk.Image[itk.RGBPixel[itk.UC], 3]
LMLOUL3 = itk.LabelMap[itk.StatisticsLabelObject[itk.UL, 3]]

# DICOM Pipeline

In [4]:
# generate dicom series file names
generator = itk.GDCMSeriesFileNames.New()
generator.SetDirectory(DICOM_PATH)
seriesUIDs = generator.GetSeriesUIDs()
series = { uid: generator.GetFileNames(uid) for uid in generator.GetSeriesUIDs() }

# imageSeriesReader
imageSeriesReader = itk.ImageSeriesReader[IF3].New()
imageSeriesReader.SetFileNames(series[seriesUIDs[2]])
imageSeriesReader.Update()

rescaleIntensityImageFilter = itk.RescaleIntensityImageFilter[IF3, IF3].New()
rescaleIntensityImageFilter.SetInput(imageSeriesReader.GetOutput())
rescaleIntensityImageFilter.SetOutputMaximum(255)
rescaleIntensityImageFilter.SetOutputMinimum(0)

castImageFilter = itk.CastImageFilter[IF3, IUC3].New()
castImageFilter.SetInput(rescaleIntensityImageFilter.GetOutput())

# NII Pipeline

In [ ]:
niiFileReader = itk.ImageFileReader[IUC3].New()
niiFileReader.SetFileName(NII_PATH)
niiFileReader.Update()

# Resample Labels to match DICOM
resampleImageFilter = itk.ResampleImageFilter[IUC3, IUC3].New()
resampleImageFilter.SetInput(niiFileReader.GetOutput())
resampleImageFilter.SetTransform(itk.IdentityTransform[itk.D, 3].New())
resampleImageFilter.SetInterpolator(itk.NearestNeighborInterpolateImageFunction[IUC3, itk.D].New())
resampleImageFilter.UseReferenceImageOn()
resampleImageFilter.SetReferenceImage(imageSeriesReader.GetOutput())
# resampleImageFilter.Update()
# resampleData = resampleImageFilter.GetOutput()

labelImageToLabelMapFilter = itk.LabelImageToLabelMapFilter[IUC3, LMLOUL3].New()
labelImageToLabelMapFilter.SetInput(resampleImageFilter.GetOutput())

# Check Physical Space

In [ ]:
# labelData = labelImageToLabelMapFilter.GetOutput()
# imageData = castImageFilter.GetOutput()
# print('label spacing', labelData.GetSpacing())
# print('label origin', labelData.GetOrigin())
# print('image spacing', imageData.GetSpacing())
# print('image spacing', imageData.GetOrigin())
# labelData.SetOrigin(labelData.GetOrigin())
# labelData.SetSpacing(labelData.GetSpacing())

# Merge DICOM and NII Pipeline

In [ ]:
labelMapOverlayImageFilter = itk.LabelMapOverlayImageFilter[LMLOUL3, IUC3, IRGBUC3].New()
labelMapOverlayImageFilter.SetInput(labelImageToLabelMapFilter.GetOutput())
labelMapOverlayImageFilter.SetFeatureImage(castImageFilter.GetOutput())
labelMapOverlayImageFilter.SetOpacity( 0.5 );

# ITK to VTK

In [ ]:
imageToVTKImageFilter = itk.ImageToVTKImageFilter[IRGBUC3].New()
imageToVTKImageFilter.SetInput(labelMapOverlayImageFilter.GetOutput())
imageToVTKImageFilter.Update()

In [ ]:
class SliceViewInteractorStyle(vtk.vtkInteractorStyleUser):
    def __init__(self, parent=None, imageViewer=None):
        self.AddObserver(vtkCommand.MouseWheelForwardEvent, self.mouseWheelForwardEvent)        
        self.AddObserver(vtkCommand.MouseWheelBackwardEvent, self.mouseWheelBackwardEvent)
        if imageViewer:
            self.imageViewer = imageViewer
            self.minSlice = imageViewer.GetSliceMin()
            self.maxSlice = imageViewer.GetSliceMax()
            self.slice = round((self.minSlice + self.maxSlice) / 2)
            print(self.minSlice, self.maxSlice)
        
    def nextSlice(self):
        if self.imageViewer and self.slice < self.maxSlice:
            self.slice = self.slice + 1
            self.imageViewer.SetSlice(self.slice)

    def previousSlice(self):
        if self.imageViewer and self.slice > self.minSlice:
            self.slice = self.slice - 1
            self.imageViewer.SetSlice(self.slice)
    
    def mouseWheelForwardEvent(self, obj, event):
        self.nextSlice()
        return
    
    def mouseWheelBackwardEvent(self, obj, event):
        self.previousSlice()
        return

imageViewer = vtk.vtkImageViewer2()
imageViewer.SetInputData(imageToVTKImageFilter.GetOutput())

renderWindow = imageViewer.GetRenderWindow()
interactor = vtk.vtkRenderWindowInteractor()
sliceInteractorStyle = SliceViewInteractorStyle(imageViewer=imageViewer)
interactor.SetInteractorStyle(sliceInteractorStyle)
interactor.SetRenderWindow(renderWindow)

renderWindow.Render()
interactor.Initialize()
interactor.Start()

In [ ]:
# itk.GetArrayFromImage(labelMapOverlayImageFilter.GetOutput())
# labelData = labelImageToLabelMapFilter.GetOutput()
# imageData = castImageFilter.GetOutput()
print('label spacing', labelData.GetSpacing())
print('label origin', labelData.GetOrigin())
print('image spacing', imageData.GetSpacing())
print('image spacing', imageData.GetOrigin())

In [5]:
out = imageSeriesReader.GetOutput()

In [7]:
d = imageSeriesReader.GetMetaDataDictionary()

In [ ]:
imageSeriesReader